In [22]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import stockdata as sd
import numpy as np

In [30]:
currdt, prevdt = sd.getdata("select distinct date from nsehistprice order by 1 desc limit 2").date.to_list()
df0 = sd.getdata(f"select date, symbol, open, low, high, close, volume from nsehistprice where date in ('{currdt}', '{prevdt}') and symbol in (select symbol from nseindices where indexname = 'Nifty 100')")
df  = sd.addpricechange(df0)
df  = sd.addvolumechange(df)
df = df[df.date == currdt]
df = df[df.prevclose!=0]
# cols = ['date', 'symbol', 'prevclose', 'open', 'low', 'high', '' ,'close', 'pricechange', 'pricechangepct', 'volume', 'prevvolume', 'volumechange', 'volumechangepct']
# df = df[cols]
df['trading range'] = df['high'] - df['low']
df['openinggap']  = df['open'] - df['prevclose']
df['openingtype'] = df['openinggap'].apply(lambda x: 'No-Gap' if x == 0 else ('Gap-Up' if x > 0 else 'Gap-Down'))
df['closinggap'] = df['close'] - df['open']
df['closingtype'] = df['closinggap'].apply(lambda x: 'No-Gap' if x == 0 else ('Above-Open' if x > 0 else 'Below-Open'))
df['volume'] = df['volume'].apply(lambda x: "{:,}".format(x))
df['prevvolume'] = df['prevvolume'].apply(lambda x: "{:,}".format(x))
df['volumechange'] = df['volumechange'].apply(lambda x: "{:,}".format(x))
df.reset_index(drop=True, inplace=True)

In [31]:
df

,date,symbol,open,low,high,close,volume,prevclose,pricechange,pricechangepct,prevvolume,volumechange,volumechangepct,trading range,openinggap,openingtype,closinggap,closingtype
0,2020-07-20,ACC,1315.85,1314.50,1334.80,1329.80,"1,539,073",1315.85,13.95,1.06,"1,179,361","359,712",30.50,20.30,0.00,No-Gap,13.95,Above-Open
1,2020-07-20,ADANIPORTS,316.00,310.35,318.70,317.30,"4,328,735",313.00,4.30,1.37,"3,264,266","1,064,469",32.61,8.35,3.00,Gap-Up,1.30,Above-Open
2,2020-07-20,ADANITRANS,242.10,239.10,247.20,240.65,"229,643",241.70,-1.05,-0.43,"257,246","-27,603",-10.73,8.10,0.40,Gap-Up,-1.45,Below-Open
3,2020-07-20,AMBUJACEM,195.00,194.50,197.00,195.70,"2,204,858",193.45,2.25,1.16,"3,254,793","-1,049,935",-32.26,2.50,1.55,Gap-Up,0.70,Above-Open
4,2020-07-20,ASHOKLEY,52.20,51.25,52.80,51.55,"31,407,348",51.75,-0.20,-0.39,"35,679,122","-4,271,774",-11.97,1.55,0.45,Gap-Up,-0.65,Below-Open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2020-07-20,ULTRACEMCO,3863.10,3826.95,3890.00,3865.10,"277,775",3862.85,2.25,0.06,"237,461","40,314",16.98,63.05,0.25,Gap-Up,2.00,Above-Open
97,2020-07-20,UPL,440.00,439.30,456.00,453.65,"9,435,204",436.45,17.20,3.94,"3,644,124","5,791,080",158.92,16.70,3.55,Gap-Up,13.65,Above-Open
98,2020-07-20,VEDL,109.35,107.65,110.40,108.10,"7,764,369",109.15,-1.05,-0.96,"9,373,147","-1,608,778",-17.16,2.75,0.20,Gap-Up,-1.25,Below-Open
99,2020-07-20,WIPRO,261.35,259.00,273.55,270.80,"21,394,990",261.35,9.45,3.62,"18,856,512","2,538,478",13.46,14.55,0.00,No-Gap,9.45,Above-Open


#### **NR4 Screener**

In [79]:
x = sd.getdata("select distinct date from nsehistprice order by 1 desc limit 4").date.to_list()
dts = "'"+ "','".join(x) + "'"
df = sd.getdata(f"select date, symbol, high, low, high-low tr from nsehistprice where date in ({dts}) and symbol in (select symbol from nseindices where indexname = 'Nifty 100')")

In [80]:
df['prevhigh'] = df.groupby('symbol').high.transform(lambda x: x.shift(1))
df['prevlow'] = df.groupby('symbol').low.transform(lambda x: x.shift(1))
df['tr-1d'] = df.groupby('symbol').tr.transform(lambda x: x.shift(1))
df['tr-2d'] = df.groupby('symbol').tr.transform(lambda x: x.shift(2))
df['tr-3d'] = df.groupby('symbol').tr.transform(lambda x: x.shift(3))
df.dropna(inplace=True)

In [81]:
df['NR4'] = df.apply(lambda x: 1 if x['tr'] < x['tr-1d'] and x['tr'] < x['tr-2d'] and x['tr'] < x['tr-3d'] else 0, axis=1) #and x['high'] < x['prevhigh'] and x['low'] > x['prevlow'] else 0, axis=1)

In [82]:
df[df.NR4==1].reset_index(drop=True)

,date,symbol,high,low,tr,prevhigh,prevlow,tr-1d,tr-2d,tr-3d,NR4
0,2020-07-21,ADANITRANS,248.40,240.65,7.75,247.20,239.10,8.10,10.10,11.90,1
1,2020-07-21,ASHOKLEY,53.15,51.80,1.35,52.80,51.25,1.55,2.10,2.15,1
2,2020-07-21,AXISBANK,451.00,438.50,12.50,442.80,429.30,13.50,15.30,17.70,1
3,2020-07-21,BANDHANBNK,356.95,352.00,4.95,358.80,347.70,11.10,13.05,24.15,1
4,2020-07-21,BANKBARODA,49.85,48.70,1.15,50.30,48.40,1.90,1.65,1.45,1
5,2020-07-21,BIOCON,433.50,424.55,8.95,434.90,424.35,10.55,10.30,16.30,1
6,2020-07-21,BRITANNIA,4010.00,3875.00,135.00,3993.00,3807.20,185.80,171.20,190.70,1
7,2020-07-21,COALINDIA,134.50,132.00,2.50,134.60,131.70,2.90,5.45,3.40,1
8,2020-07-21,DMART,2054.90,2021.05,33.85,2039.00,1968.00,71.00,64.80,169.70,1
9,2020-07-21,HCLTECH,662.35,643.30,19.05,652.00,626.85,25.15,31.25,45.05,1


In [84]:
import pandas as pd
df = pd.read_csv('https://archives.nseindia.com/content/indices/ind_niftybanklist.csv')

In [85]:
df

,Company Name,Industry,Symbol,Series,ISIN Code
0,Axis Bank Ltd.,FINANCIAL SERVICES,AXISBANK,EQ,INE238A01034
1,Bandhan Bank Ltd.,FINANCIAL SERVICES,BANDHANBNK,EQ,INE545U01014
2,Bank of Baroda,FINANCIAL SERVICES,BANKBARODA,EQ,INE028A01039
3,Federal Bank Ltd.,FINANCIAL SERVICES,FEDERALBNK,EQ,INE171A01029
4,HDFC Bank Ltd.,FINANCIAL SERVICES,HDFCBANK,EQ,INE040A01034
5,ICICI Bank Ltd.,FINANCIAL SERVICES,ICICIBANK,EQ,INE090A01021
6,IDFC First Bank Ltd.,FINANCIAL SERVICES,IDFCFIRSTB,EQ,INE092T01019
7,IndusInd Bank Ltd.,FINANCIAL SERVICES,INDUSINDBK,EQ,INE095A01012
8,Kotak Mahindra Bank Ltd.,FINANCIAL SERVICES,KOTAKBANK,EQ,INE237A01028
9,Punjab National Bank,FINANCIAL SERVICES,PNB,EQ,INE160A01022
